# Process Globe-LFMC samples and extract DEM Data
Extracts the samples for locations in the CONUS from the Globe-LFMC spread sheet and adds the normalised DEM, climate zone, and other auxiliary data. DEM data is from the GEE SRTM DEM product, extracted using the MODIS projection and scale. Sites within the same MODIS pixel are merged. The following files are created:
- `LFMC_CONUS.csv`: CONUS data extracted from the Globe-LFMC dataset
- `LFMC_sites.csv`: sites extracted from the Globe-LFMC CONUS data and augmented with normalised DEM and location data
- `LFMC_samples.csv`: Globe-LFMC CONUS sample data augmented with auxiliary variables

### Notes
1. The `Globe-LFMC-v2.xlsx` should exist in the `INPUT_DIR` directory - by default, a sub-directory of `DATA_DIR`
2. The tiff containing the Koppen climate zone data (`Beck_KG_V1_present_0p0083.tif` available from https://figshare.com/articles/dataset/Present_and_future_K_ppen-Geiger_climate_classification_maps_at_1-km_resolution/6396959/2) should also be in `INPUT_DIR`, as should either the `legend.txt` file (available from the same site) or `Climate_zones.csv`. If `Climate_zones.csv` doesn't exist, it needs to be created from `legend.txt` by uncommenting and running the first code cell under "Climate zone processing".
3. `EXTRACT_NAME` is a sub-directory of `DATA_DIR`. It will be created if it doesn't exist. All data files created by this and other data extraction notebooks will be located in sub-directories of this directory.
4. `LFMC_CONUS.csv` is created in the `INPUT_DIR` directory.
5. All other created files are CSVs and stored in the `SAMPLE_DIR` directory, by default a sub-directory of `DATA_DIR/EXTRACT_NAME`.
6. The samples data output by this code is further processed by the MODIS extraction code to remove the snow samples.

In [1]:
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime
from datetime import timedelta

import initialise
import common
from data_extract_utils import normalise_dem
from data_extract_utils import extract_koppen_data
from data_prep_utils import normalise

Define input and output files

In [9]:
# Globe-LFMC file and sheet name
GLOBE_LFMC = os.path.join(common.SOURCE_DIR, "Globe-LFMC-v2.xlsx")
SHEET_NAME = "LFMC data"
# GLOBE_LFMC = os.path.join(common.SOURCE_DIR, "Globe-LFMC.csv")

# File Names
LFMC_RAW = os.path.join(common.SOURCE_DIR, "LFMC_Europe.csv")                  # CSV of Europe data extracted from the Globe-LFMC dataset
KOPPEN_TIF = os.path.join(common.SOURCE_DIR, 'Beck_KG_V1_present_0p0083.tif')  # Tiff of Koppen climate zone values
LEGEND_FILE = os.path.join(common.SOURCE_DIR, 'legend.txt')                    # Text file with Koppen climate zone legend
KOPPEN_LEGEND = os.path.join(common.SOURCE_DIR, 'Climate_zones.csv')           # CSV of Koppen climate zone legend

SITES_OUTPUT = os.path.join(common.DATASETS_DIR, 'LFMC_europe_sites.csv')
SAMPLES_OUTPUT = os.path.join(common.DATASETS_DIR, 'LFMC_europe_samples.csv')

if not os.path.exists(common.DATASETS_DIR):
    os.makedirs(common.DATASETS_DIR)


Other constants/parameters

In [4]:
# DEM Product, projection and resolution
DEM_PRODUCT = 'USGS/SRTMGL1_003'
DEM_PROJ = "EPSG:4326"
DEM_SCALE = 30

# Floating point precision
FLOAT_PRE = 5

Initialise Google Earth Engine

In [5]:
import ee
ee.Initialize()

## Point-based Processing
Extracts the DEM data from GEE usong the native DEM projection and resolution. Keeps the sample site latitude and longitude, and adds the elevation/slope/aspect.
- Parameter:
 - sites: Dataframe of sample sites
- Returns: Dataframe of sites, latitude and longitude and the added elevation/slope/aspect attributes

In [6]:
def sites_by_point(sites):
    dem_image = ee.Terrain.products(ee.Image(DEM_PRODUCT))
    points = [ee.Geometry.Point(site.Longitude, site.Latitude) for x, site in sites.iterrows()]
    dem_col = ee.ImageCollection(dem_image)
    col_list = [dem_col.getRegion(point, DEM_SCALE, DEM_PROJ) for point in points]
    dem_list = ee.List(col_list).getInfo()
    dem_data = pd.DataFrame([item[1] for item in dem_list], columns=dem_list[0][0])
    dem_data.id = sites.Site
    dem_data.rename(columns={"id": "Site"}, inplace=True)
    dem_df = sites.merge(dem_data[['Site', 'elevation', 'slope', 'aspect']])
    dem_df.columns = ['Site', 'Latitude', 'Longitude', 'Elevation', 'Slope', 'Aspect']
    return dem_df

## Pixel-based Processing
Extracts the DEM data at the requested projection and resolution. Terrain.products adds the slope and aspect. A reducer is used so terrain product info is added before resampling.
- Parameters:
 - sites: dataframe of sampling sites
 - scale/proj: the required scale/proj (e.g. MODIS scale/proj - or map scale/proj)
 - maxPixels: Reducer parameter specifying the maximum number of DEM pixels to use to compute each down-sampled pixel. Doesn't need to be exact but make sure it's large enough - 512 is good for MODIS
- Returns: Dataframe of sites with latitude and longitude set to the pixel centroid as returned by GEE and the added elevation/slope/aspect attributes

In [7]:
def sites_by_pixel(sites, scale, proj, maxPixels):
    dem_image = ee.Terrain.products(ee.Image(DEM_PRODUCT)).reduceResolution(ee.Reducer.mean(), maxPixels=maxPixels)
    points = [ee.Geometry.Point(site.Longitude, site.Latitude) for x, site in sites.iterrows()]
    dem_col = ee.ImageCollection(dem_image)
    col_list = [dem_col.getRegion(point, scale, proj) for point in points]
    dem_list = ee.List(col_list).getInfo()
    dem_data = pd.DataFrame([item[1] for item in dem_list], columns=dem_list[0][0])
    dem_data.id = sites.Site
    dem_data.columns = ['Site', 'Longitude', 'Latitude', 'time', 'Elevation', 'Slope', 'Aspect', 'hillshade']
    dem_df = dem_data.drop(columns=["time", "hillshade"])
    return dem_df

## Main Processing
- If the LFMC_RAW file already exists, load it.
- Otherwise extract Globe LFMC data from the excel workbook sheet and save to the LFMC_RAW file.

In [11]:
if os.path.exists(LFMC_RAW):
    LFMC_data = pd.read_csv(LFMC_RAW, index_col=0, float_precision="high", parse_dates=["Sampling date"],
                           dtype={8: str, 10: np.int32, 11: np.int16, 14: np.int16, 23: str})
else:    
    LFMC_data = pd.read_excel(GLOBE_LFMC, SHEET_NAME).dropna(how="all")
    # LFMC_data = pd.read_csv(GLOBE_LFMC).dropna(how="all")
    LFMC_data = LFMC_data[(LFMC_data.Country.isin(['France', 'Italy', 'Spain']))
                          & (LFMC_data["Sampling date"] >= common.START_DATE)]
    LFMC_data.to_csv(LFMC_RAW)
    LFMC_data = LFMC_data.astype(dtype={'Sampling year': np.int32, 'Protocol': np.int16, 'Units': np.int16})
LFMC_data

,ID,Contact,Sitename,State/Region,Country,Latitude,Longitude,Sampling time,Sampling date,Sampling year,...,Units,NDVI SD min,NDVI SD max,NDVI CV min,NDVI CV max,Species collected,Elevation(m.a.s.l),Slope(%),Reference,Name of picture file
183,C2_1_1,Chuvieco,AlbAlb4,Castilla-La Mancha,Spain,38.923623,-1.717003,13:00:00,2006-04-10,2006,...,1,0.103443,0.213076,0.280429,0.440933,Pinus halepensis,NaN,NaN,"Chuvieco, E., Yebra, M., Jurdao, S., Aguado, I...",Yebra_Chinchilla_1.JPG
184,C2_1_2,Chuvieco,AlbAlb4,Castilla-La Mancha,Spain,38.923623,-1.717003,13:00:00,2006-04-10,2006,...,1,0.103443,0.213076,0.280429,0.440933,Stipa tenacissima,NaN,NaN,"Chuvieco, E., Yebra, M., Jurdao, S., Aguado, I...",NaN
185,C2_1_3,Chuvieco,AlbAlb4,Castilla-La Mancha,Spain,38.923623,-1.717003,13:40:00,2006-07-25,2006,...,1,0.103443,0.213076,0.280429,0.440933,Pinus halepensis,NaN,NaN,"Chuvieco, E., Yebra, M., Jurdao, S., Aguado, I...",Yebra_Chinchilla2 y 3.jpg
186,C2_1_4,Chuvieco,AlbAlb4,Castilla-La Mancha,Spain,38.923623,-1.717003,13:40:00,2006-07-25,2006,...,1,0.103443,0.213076,0.280429,0.440933,Stipa tenacissima,NaN,NaN,"Chuvieco, E., Yebra, M., Jurdao, S., Aguado, I...",NaN
187,C2_2_1,Chuvieco,AlbAlb5,Castilla-La Mancha,Spain,38.305241,-2.150575,16:00:00,2006-04-11,2006,...,1,0.050373,0.107793,0.102447,0.175816,Juniperus oxycedrus,NaN,NaN,"Chuvieco, E., Yebra, M., Jurdao, S., Aguado, I...",Yebra_Letur1.JPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161403,C15_1_38,Resco,Poblet,Catalonia,Spain,41.343935,1.058036,14:00:00,2017-09-26,2017,...,1,0.028424,0.074083,0.034743,0.091155,Cistus albidus,716.0,NaN,"Nolan, R.H., Hedo, J., Arteaga, C., Sugai, T. ...",NaN
161404,C15_1_39,Resco,Poblet,Catalonia,Spain,41.343935,1.058036,14:00:00,2017-09-26,2017,...,1,0.028424,0.074083,0.034743,0.091155,Rosmarinus officinalis,716.0,NaN,"Nolan, R.H., Hedo, J., Arteaga, C., Sugai, T. ...",NaN
161405,C15_1_40,Resco,Poblet,Catalonia,Spain,41.343935,1.058036,14:00:00,2017-09-26,2017,...,1,0.028424,0.074083,0.034743,0.091155,Erica arborea,716.0,NaN,"Nolan, R.H., Hedo, J., Arteaga, C., Sugai, T. ...",NaN
161406,C15_1_41,Resco,Poblet,Catalonia,Spain,41.343935,1.058036,14:00:00,2017-09-26,2017,...,1,0.028424,0.074083,0.034743,0.091155,Pinus nigra,716.0,NaN,"Nolan, R.H., Hedo, J., Arteaga, C., Sugai, T. ...",NaN


### Site processing
Extract the unique sites from the Globe-LFMC data

In [12]:
LFMC_data["Site"] = LFMC_data.ID.str.rsplit("_", 1, expand=True)[0]
sites = LFMC_data[["Site", "Latitude", "Longitude"]].drop_duplicates().reset_index(drop=True)
sites

,Site,Latitude,Longitude
0,C2_1,38.923623,-1.717003
1,C2_2,38.305241,-2.150575
2,C2_3,38.204988,-2.308259
3,C2_4,36.754853,-5.435623
4,C2_5,36.760058,-5.384230
...,...,...,...
108,C9_40,44.229000,4.020000
109,C9_41,44.275000,5.224000
110,C9_42,41.926000,8.707000
111,C12_1,40.600000,8.150000


Retrieve the DEM data from GEE - run either sitesByPixel (pixel mode) or sitesByPoint (point mode)

In [13]:
dem_df = sites_by_pixel(sites, common.SCALE, common.PROJ, 512)
dem_df

,Site,Longitude,Latitude,Elevation,Slope,Aspect
0,C2_1,-1.716492,38.922915,915.484530,14.890033,250.057486
1,C2_2,-2.153134,38.306249,974.139211,14.068808,193.215865
2,C2_3,-2.309249,38.206249,1157.154612,5.918204,96.238421
3,C2_4,-5.437234,36.756249,886.291335,22.009069,219.959201
4,C2_5,-5.385521,36.760415,1023.543996,17.130460,162.814077
...,...,...,...,...,...,...
108,C9_40,4.020829,44.227082,321.176031,18.153238,209.420276
109,C9_41,5.223086,44.277082,548.512608,19.547580,180.781877
110,C9_42,8.705806,41.927082,145.211175,18.700032,178.920620
111,C12_1,8.152258,40.602082,37.523191,5.239075,57.630698


Normalise the DEM data and save the sites data. Note: sites with same latitude/longitude are *not* merged yet

In [14]:
dem_norm = normalise_dem(dem_df.set_index('Site'), input_columns=['Longitude', 'Latitude', 'Elevation', 'Slope', 'Aspect'], precision=FLOAT_PRE)
dem_norm = dem_norm.reset_index()
dem_norm

,Site,Longitude,Latitude,Elevation,Slope,Aspect,Long_sin,Long_cos,Lat_norm,Aspect_sin,Aspect_cos
0,C2_1,-1.71649,38.92292,0.15258,0.16544,250.05749,-0.02995,0.99955,0.71624,0.94004,0.34108
1,C2_2,-2.15313,38.30625,0.16236,0.15632,193.21587,-0.03757,0.99929,0.71281,0.22862,0.97352
2,C2_3,-2.30925,38.20625,0.19286,0.06576,96.23842,-0.04029,0.99919,0.71226,-0.99408,0.10867
3,C2_4,-5.43723,36.75625,0.14772,0.24455,219.95920,-0.09476,0.99550,0.70420,0.64224,0.76650
4,C2_5,-5.38552,36.76042,0.17059,0.19034,162.81408,-0.09386,0.99559,0.70422,-0.29547,0.95535
...,...,...,...,...,...,...,...,...,...,...,...
108,C9_40,4.02083,44.22708,0.05353,0.20170,209.42028,0.07012,0.99754,0.74571,0.49121,0.87104
109,C9_41,5.22309,44.27708,0.09142,0.21720,180.78188,0.09103,0.99585,0.74598,0.01365,0.99991
110,C9_42,8.70581,41.92708,0.02420,0.20778,178.92062,0.15136,0.98848,0.73293,-0.01884,0.99982
111,C12_1,8.15226,40.60208,0.00625,0.05821,57.63070,0.14180,0.98989,0.72557,-0.84461,-0.53537


### Date processing
Create dataframe with dates and normalised day-of-year

In [15]:
days = pd.date_range(common.START_DATE, common.END_DATE, inclusive="left")
doy = pd.Series(normalise(days.dayofyear, method='range', data_range=(1, 366), scaled_range=(-np.pi, np.pi)))
days_df = pd.DataFrame({"Date": days, 
                        "Day_sin": doy.transform(np.sin).round(FLOAT_PRE),
                        "Day_cos": doy.transform(np.cos).round(FLOAT_PRE)})
days_df

C:\Users\lynnm\AppData\Local\Temp\ipykernel_16916\1585191847.py:1: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  days = pd.date_range(common.START_DATE, common.END_DATE, closed="left")


,Date,Day_sin,Day_cos
0,2000-03-01,-0.85876,-0.51237
1,2000-03-02,-0.86746,-0.49751
2,2000-03-03,-0.87589,-0.48251
3,2000-03-04,-0.88407,-0.46736
4,2000-03-05,-0.89198,-0.45207
...,...,...,...
6875,2018-12-27,0.08596,-0.99630
6876,2018-12-28,0.06880,-0.99763
6877,2018-12-29,0.05162,-0.99867
6878,2018-12-30,0.03442,-0.99941


### Climate zone processing

#### Create the Koppen legend csv file
If the `KOPPEN_LEGEND` file doesn't exist, uncomment and run the following cell. This will create it from the `legend.txt` file that can be downloaded with the climate zones tiff.

In [12]:
# legend = {}
# count = 0
# with open(LEGEND_FILE) as fp:
#     for ln in fp:
#         line = ln.split(':')
#         number = line[0].strip()
#         if number.isnumeric():
#             count += 1
#             key = int(line[0].strip())
#             parts = line[1].split('[')
#             colour = parts[1].strip().strip(']').split(' ')
#             code = parts[0].strip()[:3]
#             descr = parts[0].strip()[5:]
#             value = {'Number': number, 'Code': code, 'Description': descr, 'Red': colour[0], 'Green': colour[1], 'Blue': colour[2]}
#             legend[key] = value
# legend_df = pd.DataFrame.from_dict(legend, orient='index')
# legend_df.to_csv(KOPPEN_LEGEND, index=False)

#### Extract climate zones for sites
Extract the climate zone for each site and add to the sites data.

In [16]:
cz_columns = ['Czone1', 'Czone2', 'Czone3']
extract_koppen_data(KOPPEN_TIF, KOPPEN_LEGEND, sites, loc_columns=['Longitude', 'Latitude'], cz_columns=cz_columns)
dem_norm = dem_norm.merge(sites[['Site', 'Czone1', 'Czone2', 'Czone3']], on='Site')
dem_norm.to_csv(SITES_OUTPUT, index=False)
dem_norm

,Site,Longitude,Latitude,Elevation,Slope,Aspect,Long_sin,Long_cos,Lat_norm,Aspect_sin,Aspect_cos,Czone1,Czone2,Czone3
0,C2_1,-1.71649,38.92292,0.15258,0.16544,250.05749,-0.02995,0.99955,0.71624,0.94004,0.34108,B,BS,BSk
1,C2_2,-2.15313,38.30625,0.16236,0.15632,193.21587,-0.03757,0.99929,0.71281,0.22862,0.97352,B,BS,BSk
2,C2_3,-2.30925,38.20625,0.19286,0.06576,96.23842,-0.04029,0.99919,0.71226,-0.99408,0.10867,B,BS,BSk
3,C2_4,-5.43723,36.75625,0.14772,0.24455,219.95920,-0.09476,0.99550,0.70420,0.64224,0.76650,C,Cs,Csa
4,C2_5,-5.38552,36.76042,0.17059,0.19034,162.81408,-0.09386,0.99559,0.70422,-0.29547,0.95535,C,Cs,Csa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,C9_40,4.02083,44.22708,0.05353,0.20170,209.42028,0.07012,0.99754,0.74571,0.49121,0.87104,C,Cs,Csa
109,C9_41,5.22309,44.27708,0.09142,0.21720,180.78188,0.09103,0.99585,0.74598,0.01365,0.99991,C,Cs,Csb
110,C9_42,8.70581,41.92708,0.02420,0.20778,178.92062,0.15136,0.98848,0.73293,-0.01884,0.99982,C,Cs,Csa
111,C12_1,8.15226,40.60208,0.00625,0.05821,57.63070,0.14180,0.98989,0.72557,-0.84461,-0.53537,C,Cs,Csa


### Sample processing
Create the auxiliary dataset from the samples

##### Step 1: Merge sites and sample data to add the site longitude and latitude to the samples

In [17]:
samples = dem_norm[["Site", "Longitude", "Latitude"]].merge(
    LFMC_data[["ID", "Site", "Sampling date", "Sampling year", "Land Cover", "LFMC value"]])
samples

,Site,Longitude,Latitude,ID,Sampling date,Sampling year,Land Cover,LFMC value
0,C2_1,-1.71649,38.92292,C2_1_1,2006-04-10,2006,"Mosaic natural vegetation (tree, shrub, herbac...",89.330000
1,C2_1,-1.71649,38.92292,C2_1_2,2006-04-10,2006,"Mosaic natural vegetation (tree, shrub, herbac...",59.340000
2,C2_1,-1.71649,38.92292,C2_1_3,2006-07-25,2006,"Mosaic natural vegetation (tree, shrub, herbac...",134.620000
3,C2_1,-1.71649,38.92292,C2_1_4,2006-07-25,2006,"Mosaic natural vegetation (tree, shrub, herbac...",40.230000
4,C2_2,-2.15313,38.30625,C2_2_1,2006-04-11,2006,"Tree cover, needleleaved, evergreen, closed to...",78.180000
...,...,...,...,...,...,...,...,...
19976,C15_1,1.05726,41.34375,C15_1_38,2017-09-26,2017,"Tree cover, broadleaved, deciduous, closed to ...",61.640628
19977,C15_1,1.05726,41.34375,C15_1_39,2017-09-26,2017,"Tree cover, broadleaved, deciduous, closed to ...",60.869883
19978,C15_1,1.05726,41.34375,C15_1_40,2017-09-26,2017,"Tree cover, broadleaved, deciduous, closed to ...",60.338631
19979,C15_1,1.05726,41.34375,C15_1_41,2017-09-26,2017,"Tree cover, broadleaved, deciduous, closed to ...",103.154392


##### Step 2: Merge samples for same latitude/longitude/date

In [18]:
# Generate a common site id for each site with the same latitude and longitude
merge_columns = ["Latitude", "Longitude"]
sites_temp = dem_norm[merge_columns + ["Site"]].groupby(merge_columns, as_index=False).min()
# Merge samples for same year and location
samples = samples.merge(sites_temp, on=merge_columns, suffixes=("_x", None))
groupby_cols = ["Latitude", "Longitude", "Sampling date"]
data_cols = {"ID": "min",                                    # Unique sample ID is the first ID of the merged samples
             "Sampling year": "min",                         # They should all be the same, but need to select one
             "Land Cover": lambda x: pd.Series.mode(x)[0],   # Most common land cover value
             "LFMC value": "mean",                           # mean LFMC value
             "Site": "min"}                                  # Site id from sites_temp
samples = samples[groupby_cols + list(data_cols.keys())].groupby(groupby_cols, as_index=False).\
              agg(data_cols).round({"LFMC value": FLOAT_PRE})
samples

,Latitude,Longitude,Sampling date,ID,Sampling year,Land Cover,LFMC value,Site
0,36.75625,-5.43723,2001-06-15,C2_4_1,2001,Mosaic tree and shrub (>50%) / herbaceous cove...,140.26996,C2_4
1,36.75625,-5.43723,2001-06-30,C2_4_10,2001,Mosaic tree and shrub (>50%) / herbaceous cove...,113.64086,C2_4
2,36.75625,-5.43723,2001-07-30,C2_4_15,2001,Mosaic tree and shrub (>50%) / herbaceous cove...,103.34016,C2_4
3,36.75625,-5.43723,2001-08-25,C2_4_22,2001,Mosaic tree and shrub (>50%) / herbaceous cove...,97.21311,C2_4
4,36.76042,-5.38552,2001-06-14,C2_5_1,2001,Mosaic tree and shrub (>50%) / herbaceous cove...,142.77874,C2_5
...,...,...,...,...,...,...,...,...
9758,44.48542,6.15864,2016-06-27,C9_4_459,2016,"Tree cover, needleleaved, evergreen, closed to...",197.35104,C9_4
9759,44.48542,6.15864,2016-07-11,C9_4_461,2016,"Tree cover, needleleaved, evergreen, closed to...",169.66651,C9_4
9760,44.48542,6.15864,2016-08-01,C9_4_463,2016,"Tree cover, needleleaved, evergreen, closed to...",139.37891,C9_4
9761,44.48542,6.15864,2016-08-08,C9_4_465,2016,"Tree cover, needleleaved, evergreen, closed to...",140.19266,C9_4


##### Step 3: Add the normalised auxiliary variables (day-of-year, location and DEM) to the samples

In [19]:
aux_df = samples[["ID", "Latitude", "Longitude", "Sampling date", "Sampling year", "Land Cover", "LFMC value", "Site"]
                ].merge(days_df, left_on="Sampling date", right_on = "Date").drop(columns="Date").\
                merge(dem_norm.drop(columns=["Longitude", "Latitude"]), on="Site").sort_values("ID")
aux_df = aux_df[['ID', 'Latitude', 'Longitude', 'Sampling date', 'Sampling year', 'Land Cover', 'LFMC value', 'Site',
                 'Czone1', 'Czone2', 'Czone3',
                 'Day_sin', 'Day_cos',
                 'Long_sin', 'Long_cos', 'Lat_norm', 'Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos']]
aux_df.to_csv(SAMPLES_OUTPUT, index=False)
aux_df

,ID,Latitude,Longitude,Sampling date,Sampling year,Land Cover,LFMC value,Site,Czone1,Czone2,Czone3,Day_sin,Day_cos,Long_sin,Long_cos,Lat_norm,Elevation,Slope,Aspect_sin,Aspect_cos
9305,C12_1_1,40.60208,8.15226,2005-04-11,2005,Shrubland,151.31782,C12_1,C,Cs,Csa,-0.98868,0.15006,0.14180,0.98989,0.72557,0.00625,0.05821,-0.84461,-0.53537
9306,C12_1_10,40.60208,8.15226,2005-04-19,2005,Shrubland,156.42855,C12_1,C,Cs,Csa,-0.95872,0.28436,0.14180,0.98989,0.72557,0.00625,0.05821,-0.84461,-0.53537
9323,C12_1_100,40.60208,8.15226,2005-09-05,2005,Shrubland,69.59390,C12_1,C,Cs,Csa,0.89584,0.44438,0.14180,0.98989,0.72557,0.00625,0.05821,-0.84461,-0.53537
9324,C12_1_101,40.60208,8.15226,2005-09-13,2005,Shrubland,84.49750,C12_1,C,Cs,Csa,0.94836,0.31719,0.14180,0.98989,0.72557,0.00625,0.05821,-0.84461,-0.53537
9325,C12_1_106,40.60208,8.15226,2005-09-20,2005,Shrubland,108.69886,C12_1,C,Cs,Csa,0.97961,0.20089,0.14180,0.98989,0.72557,0.00625,0.05821,-0.84461,-0.53537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2616,C9_9_699,42.91042,2.79610,2016-08-22,2016,Shrubland,61.93016,C9_9,C,Cs,Csa,0.77488,0.63210,0.04878,0.99881,0.73839,0.06178,0.07376,0.20062,0.97967
2617,C9_9_701,42.91042,2.79610,2016-08-29,2016,Shrubland,64.65717,C9_9,C,Cs,Csa,0.84525,0.53437,0.04878,0.99881,0.73839,0.06178,0.07376,0.20062,0.97967
2618,C9_9_703,42.91042,2.79610,2016-09-05,2016,Shrubland,53.53476,C9_9,C,Cs,Csa,0.90336,0.42889,0.04878,0.99881,0.73839,0.06178,0.07376,0.20062,0.97967
2619,C9_9_705,42.91042,2.79610,2016-09-12,2016,Shrubland,50.03452,C9_9,C,Cs,Csa,0.94836,0.31719,0.04878,0.99881,0.73839,0.06178,0.07376,0.20062,0.97967
